Starter Pack

In [108]:
import os
import pandas as pd

In [109]:
dataframes = {}
folder = r'C:\Users\MATHEUS\Desktop\Pessoal\Competicoes\Kaggle\Big Data Bowl'

for filename in os.listdir(folder):
    if filename.endswith(".csv"):
        full_path= os.path.join(folder,filename)
        df = pd.read_csv(full_path)
        dataframes[filename.removesuffix('.csv')]=df

In [110]:
indexlist = ['gameId','playId','nflId']
for key in dataframes.keys():
    existingindex = [col for col in dataframes[key].columns if col in indexlist]
    dataframes[key].set_index(existingindex, inplace=True)

In [111]:
for i in range(1,10):
    dataframes['tracking_players_{}'.format(i)]=dataframes['tracking_week_{}'.format(i)].query('displayName != "football"')
    dataframes['tracking_football_{}'.format(i)]=dataframes['tracking_week_{}'.format(i)].query('displayName == "football"')
    dataframes.pop('tracking_week_{}'.format(i))

In [112]:
pos_dict = dataframes['players']['position'].to_dict()
display_dict = dataframes['players']['displayName'].to_dict()

dataframes['player_play']['position'] = dataframes['player_play'].index.get_level_values('nflId').map(pos_dict)
dataframes['player_play']['displayName'] = dataframes['player_play'].index.get_level_values('nflId').map(display_dict)


In [113]:
#Add what penalty happened during a play in a game
#Helps track if the player play was helped by penalty
dataframes['player_play']['penaltyInPlay'] = dataframes['player_play'].groupby(['gameId', 'playId'])['penaltyNames'].transform('first')


Receivers Code

In [50]:
#Define the columns to visualize
standardplayerplaywr = ['position','displayName','teamAbbr','hadPassReception','receivingYards','wasTargettedReceiver','yardageGainedAfterTheCatch','penaltyInPlay','inMotionAtBallSnap','shiftSinceLineset','motionSinceLineset','wasRunningRoute','routeRan']

Tracking Foward Pass (Pass travelled more than 5 yards)

In [94]:
fowardpassindex = dataframes['plays'].query('passLength >= 5').index
def isFowardPass(df):
    havefowardpass = df.index.droplevel('nflId').isin(fowardpassindex)\
                         & (df['wasTargettedReceiver']==1)
    return havefowardpass

Tracking Separation Fowls: Ineligble Man Downfield, OPI, Receiver Holding, Illegal hands to the face, Illegal motion, Offense Offside, Facemask


In [98]:
#Track if the pass travelled more than 5 yards and was helped with penalty
def penaltyInterferenceReceiver(df):
    penaltyList = ['Ineligible Downfield Pass','Offensive Pass Interference', 'Illegal Shift']
    fowlreviewplay = dataframes['player_play'][dataframes['player_play']['penaltyNames'].isin(penaltyList)]
    fowlreviewindex = fowlreviewplay.droplevel('nflId').index
    haveInterference = df.index.droplevel('nflId').isin(fowlreviewindex)\
                          & df.index.get_level_values('nflId').isin(receivers_ids)\
                          & (df['hadPassReception']==1)\
                          & isFowardPass(df)
    return haveInterference

Receivers with at least n targets

In [20]:
def n_targetreceivers(n):    
    receivers_playdf = dataframes['player_play'].groupby('nflId')
    receivers_play = receivers_playdf['wasTargettedReceiver'].sum()[receivers_playdf['wasTargettedReceiver'].sum() >= n]
    receivers_ids = list(receivers_play.reset_index()['nflId'])
    return receivers_ids

In [99]:
#Set n
n=10

#Set conditions
playerplayntgt = dataframes['player_play'][dataframes['player_play'].index.get_level_values('nflId').isin(n_targetreceivers(n))]
playerplayntgt[~penaltyInterferenceReceiver(playertgtntgt) & isFowardPass(playertgtntgt)][standardplayerplaywr]

position       displayName teamAbbr  hadPassReception  \
gameId     playId nflId                                                         
2022090800 56     42489       WR      Stefon Diggs      BUF                 1   
           167    42489       WR      Stefon Diggs      BUF                 1   
           236    52536       WR        Gabe Davis      BUF                 1   
           364    44881       WR       Cooper Kupp       LA                 1   
           486    44881       WR       Cooper Kupp       LA                 1   
...                          ...               ...      ...               ...   
2022110700 3299   53098       WR  Marquez Callaway       NO                 1   
           3347   46160       WR    Tre'Quan Smith       NO                 1   
           3401   46160       WR    Tre'Quan Smith       NO                 1   
           3458   53098       WR  Marquez Callaway       NO                 0   
           3480   52942       TE     Juwan Johnson       NO                 1   

                         receivingYards  wasTargettedReceiver  \
gameId     playId nflId                                         
2022090800 56     42489               6                     1   
           167    42489              12                     1   
           236    52536              26                     1   
           364    44881              23                     1   
           486    44881               6                     1   
...                                 ...                   ...   
2022110700 3299   53098               5                     1   
           3347   46160               7                     1   
           3401   46160               9                     1   
           3458   53098               0                     1   
           3480   52942              41                     1   

                         yardageGainedAfterTheCatch penaltyInPlay  \
gameId     playId nflId                                             
2022090800 56     42489                           1          None   
           167    42489                           6          None   
           236    52536                          11          None   
           364    44881                           1          None   
           486    44881                           0          None   
...                                             ...           ...   
2022110700 3299   53098                           0          None   
           3347   46160                           2          None   
           3401   46160                           1          None   
           3458   53098                           0          None   
           3480   52942                          30          None   

                        inMotionAtBallSnap shiftSinceLineset  \
gameId     playId nflId                                        
2022090800 56     42489              False             False   
           167    42489              False             False   
           236    52536               True             False   
           364    44881              False             False   
           486    44881              False             False   
...                                    ...               ...   
2022110700 3299   53098              False             False   
           3347   46160              False             False   
           3401   46160              False             False   
           3458   53098              False             False   
           3480   52942              False             False   

                        motionSinceLineset  wasRunningRoute routeRan  
gameId     playId nflId                                               
2022090800 56     42489               True              1.0       IN  
           167    42489              False              1.0    SLANT  
           236    52536               True              1.0   CORNER  
           364    44881               

OL and DL code

In [101]:
#Define columns to visualize
standardOLplayerplay = ['position','displayName','teamAbbr','penaltyNames','penaltyInPlay','blockedPlayerNFLId1','blockedPlayerNFLId2','blockedPlayerNFLId3','pressureAllowedAsBlocker','timeToPressureAllowedAsBlocker']
standardDLplayerplay = ['position','displayName','teamAbbr','penaltyNames','penaltyInPlay','forcedFumbleAsDefense','halfSackYardsAsDefense','passDefensed','quarterbackHit','sackYardsAsDefense','safetyAsDefense','wasInitialPassRusher','causedPressure','timeToPressureAsPassRusher','getOffAsPassRusher']

In [136]:
#Find Plays that have passes
passplayindex = dataframes['plays'][dataframes['plays']['isDropback']==True].index

In [135]:
#Find OL player plays
isblocker = ~dataframes['player_play']['pressureAllowedAsBlocker'].isna()
dataframes['player_play'][isblocker & dataframes['player_play'].index.droplevel('nflId').isin(passplayindex) & (dataframes['player_play']['position']=='TE')][standardOLplayerplay]
#dataframes['player_play'][dataframes['player_play'].index.droplevel('nflId').isin(passplayindex)][standardOLplayerplay]

position    displayName teamAbbr penaltyNames  \
gameId     playId nflId                                                 
2022090800 122    47879       TE    Dawson Knox      BUF          NaN   
           236    47879       TE    Dawson Knox      BUF          NaN   
                  48011       TE  Tommy Sweeney      BUF          NaN   
           617    47879       TE    Dawson Knox      BUF          NaN   
           867    47879       TE    Dawson Knox      BUF          NaN   
...                          ...            ...      ...          ...   
2022110700 1338   47852       TE    Josh Oliver      BAL          NaN   
           2412   52513       TE  Adam Trautman       NO          NaN   
                  52942       TE  Juwan Johnson       NO          NaN   
           2843   52513       TE  Adam Trautman       NO          NaN   
                  52942       TE  Juwan Johnson       NO          NaN   

                        penaltyInPlay  blockedPlayerNFLId1  \
gameId     playId nflId                                      
2022090800 122    47879          None                  NaN   
           236    47879          None              41239.0   
                  48011          None              43335.0   
           617    47879          None              43298.0   
           867    47879          None                  NaN   
...                               ...                  ...   
2022110700 1338   47852          None              44871.0   
           2412   52513          None              41363.0   
                  52942          None              41363.0   
           2843   52513          None                  NaN   
                  52942          None              37145.0   

                         blockedPlayerNFLId2  blockedPlayerNFLId3  \
gameId     playId nflId                                             
2022090800 122    47879                  NaN                  NaN   
           236    47879                  NaN                  NaN   
                  48011              47939.0                  NaN   
           617    47879                  NaN                  NaN   
           867    47879                  NaN                  NaN   
...                                      ...                  ...   
2022110700 1338   47852                  NaN                  NaN   
           2412   52513                  NaN                  NaN   
                  52942                  NaN                  NaN   
           2843   52513                  NaN                  NaN   
                  52942                  NaN                  NaN   

                         pressureAllowedAsBlocker  \
gameId     playId nflId                             
2022090800 122    47879                       0.0   
           236    47879                       0.0   
                  48011                       0.0   
           617    47879                       0.0   
           867    47879                       0.0   
...                                           ...   
2022110700 1338   47852                       1.0   
           2412   52513                       0.0   
                  52942                       0.0   
           2843   52513                       0.0   
                  52942                       0.0   

                         timeToPressureAllowedAsBlocker  
gameId     playId nflId                                  
2022090800 122    47879                             NaN  
           236    47879                             NaN  
                  48011                             NaN  
           617    47879                             NaN  
           867    47879                             NaN  
...                                                 ...  
2022110700 1338   47852                             5.0  
           2412   52513                             NaN  
                  52942                             NaN  
           2843   52513                  

In [ ]:
#Find DL player plays
